Following document and takes what I need.

# Creating the Hamiltonian

In [4]:
# Single ground state and an excited state with some detuning 
import numpy as np
import pylcp

delta = 1
mass = 1
Hg = np.array([[0.]]) # create 2 dimension array (since Hamiltonian is matrix of submatrix) but contain 1x1 matrix now since we're dealing 
He = np.array([[-delta]])
# mu, d are defined as spherical harmonics.
# It defined as 3 dimension, 1x1 matrix
# first element being the q = -1, the second q = 0, and the third q = 1
mu_q = np.zeros((3,1,1))
d_q = np.zeros((3,1,1))
d_q[1,0,0] = 1.

# There are shpae requirement.
hamiltonian = pylcp.hamiltonian(Hg, He, mu_q, mu_q, d_q, mass=mass)


# Laser beams

In [5]:
norm_intensity = 1 # ratio of saturation power.
laserBeams = pylcp.laserBeams([
        {'kvec':np.array([1., 0., 0.]), 'pol':np.array([0., 1., 0.]),
         'pol_coord':'spherical', 'delta':delta, 's':norm_intensity},
        {'kvec':np.array([-1., 0., 0.]), 'pol':np.array([0., 1., 0.]),
         'pol_coord':'spherical', 'delta':delta, 's':norm_intensity}
        ], beam_type=pylcp.infinitePlaneWaveBeam)

# Magnetic field and Governing equation

In [7]:
alpha = 1 # gradient ratio
magField = pylcp.quadrupoleMagneticField(alpha)
obe = pylcp.obe(laserBeams, magField, hamiltonian)
v = np.arange(-10.,10.1, 0.1)
profile = obe.generate_force_profile(np.zeros((3,)+v.shape), [v, np.zeros(v.shape), np.zeros(v.shape)])

# $F=2\rightarrow F=3$ 1D molasses


In [8]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import pylcp
import scipy.constants as cts
from pylcp.common import progressBar
import lmfit


In [9]:
# Define the problem
atom = pylcp.atom('87Rb')
# transition.k is in cm
# dimensionless mass is gamma * M / k^2 * hbar
# state[2] is D2 line
# transition[1] corresponds to D2 transition.
# We're dealing with manifold now.
mass = (atom.state[2].gamma*atom.mass)/(cts.hbar*(100*2*np.pi*atom.transition[1].k)**2)
print(mass)

804.3755599734405


In [10]:
# Here the document create methods to return the hamiltonian and the lasers in order to sweep their parameters.
# Note that the magnetic field is always zero
# They make a dictionary of different polarizations that they will expolore.
def return_hamiltonian(Fl, Delta):
    # .singleF construct the hamiltonian for a lonely angular momentum state.
    # This works becuase we're dealing with optical molasses? basically no magnetic field.-> degenerate
    # In this example, we don't need to set gF, but just for physical correctnnes
    Hg, Bgq = pylcp.hamiltonians.singleF(F=Fl, gF=0, muB=1)
    He, Beq = pylcp.hamiltonians.singleF(F=Fl+1, gF=1/(Fl+1), muB=1)
    dijq = pylcp.hamiltonians.dqij_two_bare_hyperfine(Fl, (Fl+1))
    # Note how to use np.eye
    hamiltonian = pylcp.hamiltonian(Hg, -Delta*np.eye(He.shape[0])+He, Bgq, Beq, dijq, mass = mass)
    
    return hamiltonian

# Define 1D laser beams
def return_lasers(delta, s, pol):
    # Look at the pol setup below (defined dictionary)
    # Here, we set counter propagating light configuration.
    if pol[0][2]>0 or pol[0][1]>0:
        pol_coord = 'spherical'
    else:
        pol_coord = 'cartesian'
    return pylcp.laserBeams([
            {'kvec':np.array([0., 0., 1.]), 'pol':pol[0],
             'pol_coord':pol_coord, 'delta':delta, 's':s},
            {'kvec':np.array([0., 0., -1.]), 'pol':pol[1],
             'pol_coord':pol_coord, 'delta':delta, 's':s},
            ], beam_type=pylcp.infinitePlaneWaveBeam)

pols = {'$\\sigma^+\\sigma^-$':[np.array([0.,0.,1.]), np.array([1., 0., 0.])],
        '$\\sigma^+\\sigma^+$':[np.array([0., 0., 1.]), np.array([0., 0., 1.])]}

# This line is for adding counter-linear polarization setup.
# Note that in this case, laserbeam configuration is cartesian (along x-axis).
phi = [0, np.pi/4, np.pi/2]
phi_keys = ['$\\phi=0$', '$\\phi=\\pi/4$', '$\\phi=\\pi/2$']
for phi_i, key_beam in zip(phi, phi_keys):
        pols[key_beam] = [np.array([1., 0., 0.]), np.array([np.cos(phi_i), np.sin(phi_i), 0.])]
    

In [12]:
det = -2.5
s = 1.0

hamiltonian = return_hamiltonian(2, det)

v = np.concatenate((np.arange(0.001, 0.01, 0.001),
                    np.arange(0.01, 0.02, 0.002),
                    np.arange(0.02, 0.03, 0.005),
                    np.arange(0.03, 0.1, 0.01),
                    np.arange(0.1, 5.1, 0.1)))

obe = {}
for key_beam in pols:
    laserBeams = return_lasers(0., s, pol=pols[key_beam])

    obe[key_beam] = pylcp.obe(
        laserBeams, magField, hamiltonian,
        include_mag_forces=False, transform_into_re_im=True
    )

    obe[key_beam].generate_force_profile(
        [np.zeros(v.shape), np.zeros(v.shape), np.zeros(v.shape)],
        [np.zeros(v.shape), np.zeros(v.shape), v],
        name='molasses', deltat_v=4, deltat_tmax=2*np.pi*5000, itermax=1000,
        rel=1e-8, abs=1e-10, progress_bar=True
    )

KeyboardInterrupt: 

In [ ]:
fig, ax = plt.subplots(1, 1)
axins = inset_axes(ax, width=1.0, height=0.8)
for key in obe:
    if 'phi' in key:
        linestyle='--'
    else:
        linestyle='-'

    ax.plot(np.concatenate((-v[::-1], v)),
            np.concatenate((-obe[key].profile['molasses'].F[2][::-1],
                            obe[key].profile['molasses'].F[2])),
            label=pols[key], linestyle=linestyle,
            linewidth=0.75
            )
    axins.plot(np.concatenate((-v[::-1], v)),
               np.concatenate((-obe[key].profile['molasses'].F[2][::-1],
                               obe[key].profile['molasses'].F[2])),
            label=pols[key], linestyle=linestyle,
            linewidth=0.75
            )

ax.set_xlim(-5, 5)
ax.set_xlabel('$v/(\Gamma/k)$')
ax.set_ylabel('$f/(\hbar k \Gamma)$')
ax.legnd()
axins.set_xlim(-0.1, 0.1)
axins.set_ylim(-0.025, 0.025);